### Fail eeltreenimise protsessi testimiseks

### Praeguseks on "korpus" kaustas .tsv failid tekitatud ehk script korpusest_tsv_andmete_loomine.py on jooksutatud. Kuna need on üle 2GB suured, ei saa neid GitHubi laadida. Näitena on kaustas "test.tsv" fail

In [1]:
# Impordid
import os
import csv
from datasets import load_dataset
from src.transformers import BertTokenizer, BertForMaskedLM, BertConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [2]:
input_folder = "korpus"
input_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename[-3:] == "tsv"]
input_files

['korpus\\test.tsv']

In [3]:
# Võtab iga batchi (tekstid korpusest, kus laused on eraldatud \n sümboliga, 'text' : lause1 \n lause2 \n lause3)
# Tokenieerib laused ja lisab batchile labelid
# Väjund (x tähistab kõikide lausete arvu loetud batchi tekstides)
# {'input_ids': x*128*2 tensor, 'attention_mask': x*128 tensor, 'token_type_ids': x*128 tensor, 'labels': x*128*2 tensor}
def tokeniseeri_batch(batch):
    batch_laused = [lause for para in batch["text"] for lause in para.split("\n")]
    tokeniseeritud = tokenizer(batch_laused, max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")
    tokeniseeritud["labels"] = tokeniseeritud["input_ids"]
    return tokeniseeritud

# Tokeniseerija
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt", max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")

for filename in input_files:
    print(filename)
    train_dataset = load_dataset("csv", data_files={'train': filename}, names = ["text"], delimiter = "\t")["train"]
    train_dataset_sisend = train_dataset.map(tokeniseeri_batch, batched=True, remove_columns=["text"])
    # train_dataset_sisend.set_format(type ='torch')
    out_name = filename[:-4] + ".json"
    train_dataset_sisend.to_json(out_name)
    

korpus\test.tsv


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt", max_length = 128, 
                          padding = "max_length", truncation = True, return_tensors = "pt", mask_token = "ˇMASKˇ")

In [5]:
treening_andmed = load_dataset("json", data_files={'train': ["korpus/test.json"]})["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:/Users/rauln/.cache/huggingface/datasets/json/default-af143b59aca45e9b/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

In [7]:
# Mudeli eeltreenimine

config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

model = BertForMaskedLM(config)

training_args = TrainingArguments(
    output_dir='./train_results',
    per_device_train_batch_size=32,
    max_steps=10,
    learning_rate=1e-4,
    logging_dir='./train_logs',
    logging_steps=5, 
    save_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator = data_collator,
    train_dataset=treening_andmed
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 62
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 10
  Number of trainable parameters = 160300708


Step,Training Loss
5,14.011200
10,13.169300


Saving model checkpoint to ./train_results\checkpoint-10
Configuration saved in ./train_results\checkpoint-10\config.json
Model weights saved in ./train_results\checkpoint-10\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=10, training_loss=13.590219879150391, metrics={'train_runtime': 264.3772, 'train_samples_per_second': 1.21, 'train_steps_per_second': 0.038, 'total_flos': 59133040005120.0, 'train_loss': 13.590219879150391, 'epoch': 5.0})

### Eeltreenitud mudeli laadimine ja hindamine/ennustamine

In [9]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    
    indeksid = np.where(labels[:, :, 0].flatten() != -100)[0]
    
    labels_lemma = labels[:, :, 0].flatten()[indeksid]
    labels_vorm = labels[:, :, 1].flatten()[indeksid]

    pred_lemma = np.take(np.argmax(pred[0], axis = 2).flatten(), indeksid)
    pred_vorm = np.take(np.argmax(pred[1], axis = 2).flatten(), indeksid)

    accuracy_lemma = accuracy_score(y_true=labels_lemma, y_pred=pred_lemma)
    accuracy_vorm = accuracy_score(y_true=labels_vorm, y_pred=pred_vorm)

    return {"accuracy_lemma": accuracy_lemma, "accuracy_vorm": accuracy_vorm}

model = BertForMaskedLM.from_pretrained("train_results\checkpoint-10")

training_args = TrainingArguments(
    output_dir='./eval_results',
    per_device_eval_batch_size=32,
    max_steps=10,
    eval_steps = 10
)

trainer = Trainer(
    model=model,
    args = training_args,
    data_collator = data_collator,
    eval_dataset = train_dataset_sisend,
    compute_metrics = compute_metrics
)

trainer.evaluate()

loading configuration file train_results\checkpoint-10\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "hidden_size_form": 48,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 128,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.25.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50005,
  "vocab_size_form": 111
}

loading weights file train_results\checkpoint-10\pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at train_r

{'eval_loss': 13.126060485839844,
 'eval_accuracy_lemma': 0.046511627906976744,
 'eval_accuracy_vorm': 0.3488372093023256,
 'eval_runtime': 15.7585,
 'eval_samples_per_second': 3.934,
 'eval_steps_per_second': 0.127}

In [12]:
pred, labels, _ = trainer.predict(train_dataset_sisend)
    
indeksid = np.where(labels[:, :, 0].flatten() != -100)[0]
    
labels_lemma = labels[:, :, 0].flatten()[indeksid]
labels_vorm = labels[:, :, 1].flatten()[indeksid]

pred_lemma = np.take(np.argmax(pred[0], axis = 2).flatten(), indeksid)
pred_vorm = np.take(np.argmax(pred[1], axis = 2).flatten(), indeksid)

accuracy_lemma = accuracy_score(y_true=labels_lemma, y_pred=pred_lemma)
accuracy_vorm = accuracy_score(y_true=labels_vorm, y_pred=pred_vorm)

print(f"accuracy_lemma: {accuracy_lemma}")
print(f"accuracy_vorm: {accuracy_vorm}")

***** Running Prediction *****
  Num examples = 62
  Batch size = 32


accuracy_lemma: 0.03205128205128205
accuracy_vorm: 0.3782051282051282


In [13]:
pred_lemma

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5], dtype=int64)

In [15]:
labels_lemma

array([13226,   495,     9,  2818,   481,  9807,  3001,  1323,    92,
         303,   131,    52,    89,   418, 15864,   119,     6, 19765,
        9874,  1324,   685,    77,   938,     6,   174,   161,  2051,
           6,   113,  5376,   296,   917,   534,   596, 38325,  1571,
         570,     5,     5,    10, 23473,    55,     6, 45401,    20,
         385,    10,    55,  4297,    45,   647,    13,   263,   230,
          15,  3068,   408, 23473,  2239,    75,  3068, 23473,   124,
         796,   125,   407,   168,     6,   258,  1236,    21,   107,
           6,   521,     7,    17,   899,   240,    19,    21,   682,
           6,  1188,   321,    10,  1309,     5,  1551,    11,  1551,
          12,  2316,    55,    32,  1224,  1224,   164,   473,   360,
          14,    92,  2303,  7787, 11066,    25,   360,   840,    32,
          24,  1083,   287,   305, 11264,    13,    31,    27,   266,
           7,    84,   206,     5,    18,    46,     9,    16,  2291,
        2176,   249,

In [14]:
pred_vorm

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5], dtype=int64)

In [16]:
labels_vorm

array([  5,  31,  31,  31,  56,  35,  35,  35,   5,  56,  29,   5,  56,
        26,  35,   5,   5,  31,  33,  26,  31,  35,  37,   5,  31,   5,
        26,   5,   5,  35,   5,  51,   5,  50,   6,  54,  36,   5,   5,
         5,  36,   5,   5,  46,   5,  91,   5,   5,   6, 104,  31,  41,
        26,  31,   5,  35,   5,  46,  36,  26,  31,  50,   5,  83,   5,
        35,  90,   5,  29,   6,   5,   5,   5,   5,  54,   5,  31,  31,
         5,   5,  26,   5,  56,  35,   5,  36,   5,  35,  31,  35,  72,
        50,   5,   5,  35,  31,   5,  61,  31,   5,   5,  35,  31,  38,
        29,  31,  31,   5,   5,  31,   5,  44,  29,  35,  56,   5,  54,
        54,  31,   5,   5,  96,   5,  46,  31,  36,  51,  51,   5,  43,
        56,  51,  56, 109,   5,   5,   5,  31,   5,  62,   5,  66,  50,
         5,   5,  54,   5,   5,  38,   5,  72,  90,   5,   5,  56,  72],
      dtype=int64)